# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f55e8018790>
├── 'a' --> tensor([[-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298]])
└── 'x' --> <FastTreeValue 0x7f55e8077fd0>
    └── 'c' --> tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                        [-1.0355,  0.5128,  0.6492,  0.7051],
                        [ 0.1372, -0.6870, -0.3957,  0.6132]])

In [4]:
t.a

tensor([[-0.4370, -0.9170,  0.4492],
        [-0.4437,  1.8622,  0.7298]])

In [5]:
%timeit t.a

63.8 ns ± 0.0829 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f55e8018790>
├── 'a' --> tensor([[-0.2494, -1.0558,  0.6897],
│                   [-0.8364, -0.4471,  0.3607]])
└── 'x' --> <FastTreeValue 0x7f55e8077fd0>
    └── 'c' --> tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                        [-1.0355,  0.5128,  0.6492,  0.7051],
                        [ 0.1372, -0.6870, -0.3957,  0.6132]])

In [7]:
%timeit t.a = new_value

66.3 ns ± 0.0994 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298]]),
    x: Batch(
           c: tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                      [-1.0355,  0.5128,  0.6492,  0.7051],
                      [ 0.1372, -0.6870, -0.3957,  0.6132]]),
       ),
)

In [10]:
b.a

tensor([[-0.4370, -0.9170,  0.4492],
        [-0.4437,  1.8622,  0.7298]])

In [11]:
%timeit b.a

60.1 ns ± 0.0143 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8329, -2.4000,  0.1343],
               [-0.7825, -0.9174,  0.3500]]),
    x: Batch(
           c: tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                      [-1.0355,  0.5128,  0.6492,  0.7051],
                      [ 0.1372, -0.6870, -0.3957,  0.6132]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.0644 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

809 ns ± 0.0881 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 18.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 960 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 515 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f55332d6a90>
├── 'a' --> tensor([[[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]],
│           
│                   [[-0.4370, -0.9170,  0.4492],
│                    [-0.4437,  1.8622,  0.7298]]])
└── 'x' --> <FastTreeValue 0x7f55332d67f0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.6 µs ± 47.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f554498bfd0>
├── 'a' --> tensor([[-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298],
│                   [-0.4370, -0.9170,  0.4492],
│                   [-0.4437,  1.8622,  0.7298]])
└── 'x' --> <FastTreeValue 0x7f55448f2520>
    └── 'c' --> tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                        [-1.0355,  0.5128,  0.6492,  0.7051],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 53.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.2 µs ± 159 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]],
       
               [[-0.4370, -0.9170,  0.4492],
                [-0.4437,  1.8622,  0.7298]]]),
    x: Batch(
           c: tensor([[[ 0.5381,  0.0749, -0.1468,  0.4915],
                       [-1.0355,  0.5128,  0.6492,  0.7051],
                       [ 0.1372, -0.6870, -0.3957,  0.6132]],
         

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 865 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298],
               [-0.4370, -0.9170,  0.4492],
               [-0.4437,  1.8622,  0.7298]]),
    x: Batch(
           c: tensor([[ 0.5381,  0.0749, -0.1468,  0.4915],
                      [-1.0355,  0.5128,  0.6492,  0.7051],
                      [ 0.1372, -0.6870, -0.3957,  0.6132],
                      [ 0.5381,  0.0749, -0.1468,  0.4915],
                      [-1.0355,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 258 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

324 µs ± 728 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
